In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
!pip install fuzzywuzzy
!pip install sentence_transformers
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb
import nltk
import time
from matplotlib.pyplot import figure
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
# import optuna
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import warnings
import gc
from sklearn.model_selection import cross_val_score
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

     |████████████████████████████████| 79 kB 5.6 MB/s 
     |████████████████████████████████| 4.0 MB 45.3 MB/s 
     |████████████████████████████████| 1.2 MB 30.5 MB/s 
     |████████████████████████████████| 77 kB 4.0 MB/s 
     |████████████████████████████████| 596 kB 44.6 MB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 6.5 MB 42.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=e4550c5b2dbed9b5fe2b51f4975962ef69e227cfd8e060af9655fce0c327c75f
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive/Quora-Questions/"

Mounted at /content/drive
 new_boolq.jsonl   quora_train.csv  'test (1).csv'   train.csv.zip
 quora_test.csv    results	     test.csv


In [62]:
# data = pd.read_csv('/content/drive/My Drive/Quora-Questions/train.csv.zip')
train = pd.read_csv('/content/drive/My Drive/Quora-Questions/quora_train.csv')
test = pd.read_csv('/content/drive/My Drive/Quora-Questions/quora_test.csv')
y_train = train.is_duplicate
y_test = test.is_duplicate
train = train.drop(columns=['is_duplicate'])
test = test.drop(columns=['is_duplicate'])


In [4]:
from zeugma.embeddings import EmbeddingTransformer
glove = EmbeddingTransformer('glove')

     |████████████████████████████████| 462 kB 13.1 MB/s 
  Created wheel for zeugma: filename=zeugma-0.49-py3-none-any.whl size=8823 sha256=d9d6068cabe1262910f43dd2d3197954f3ad1bdb3eb8187edb758a7e3f556b7e
  Stored in directory: /root/.cache/pip/wheels/1d/47/5b/2a59a79706cc9340c72fd6a7bfc20e7ebcab849c88c38fdfa0
Successfully built zeugma
[==================================================] 100.0% 104.8/104.8MB downloaded


In [63]:
train_question1 = train[['question1_final']].to_numpy().tolist()
train_question1 = [item for sublist in train_question1 for item in sublist]

In [64]:
train_question1 = [" " if type(x) == float else x for x in train_question1]

In [65]:
train_question1_vec = glove.transform(train_question1)

In [66]:
train_question2 = train[['question2_final']].to_numpy().tolist()
train_question2 = [item for sublist in train_question2 for item in sublist]
train_question2 = [" " if type(x) == float else x for x in train_question2]

In [67]:
train_question2_vec = glove.transform(train_question2)

In [68]:
train = train.drop(columns=['question1_final', 'question2_final'])
train = np.hstack((train_question1_vec, train_question2_vec))
train.shape

(283003, 50)

In [69]:
splits = ShuffleSplit(n_splits=1, test_size=.3, random_state=42)
svc_param_grid = {'C':[0.01, 0.1, 1, 10, 100]}
svc_clf = LinearSVC(penalty='l2', loss='squared_hinge', dual=False, max_iter=3000)
svc_clf_search = HalvingGridSearchCV(svc_clf, svc_param_grid, cv=splits, factor=2, scoring='accuracy', verbose=3)

In [70]:
svc_clf_search.fit(train, y_train)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 70750
max_resources_: 283003
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 5
n_resources: 70750
Fitting 1 folds for each of 5 candidates, totalling 5 fits
[CV 1/1] END ........C=0.01;, score=(train=0.643, test=0.644) total time=   0.9s
[CV 1/1] END .........C=0.1;, score=(train=0.643, test=0.643) total time=   1.2s
[CV 1/1] END ...........C=1;, score=(train=0.643, test=0.644) total time=   1.2s
[CV 1/1] END ..........C=10;, score=(train=0.643, test=0.644) total time=   1.2s
[CV 1/1] END .........C=100;, score=(train=0.643, test=0.644) total time=   1.2s
----------
iter: 1
n_candidates: 3
n_resources: 141500
Fitting 1 folds for each of 3 candidates, totalling 3 fits
[CV 1/1] END .........C=100;, score=(train=0.647, test=0.646) total time=   2.2s
[CV 1/1] END ...........C=1;, score=(train=0.647, test=0.646) total time=   2.2s
[CV 1/1] END ........C=0.01;, score=(train=0.646, test=

HalvingGridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=42, test_size=0.3, train_size=None),
                    estimator=LinearSVC(dual=False, max_iter=3000), factor=2,
                    param_grid={'C': [0.01, 0.1, 1, 10, 100]},
                    scoring='accuracy', verbose=3)

In [71]:
svc_clf_model =  svc_clf_search.best_estimator_
svc_clf_model

LinearSVC(C=100, dual=False, max_iter=3000)

In [72]:
test_question1 = test[['question1_final']].to_numpy().tolist()
test_question1 = [item for sublist in test_question1 for item in sublist]
test_question1 = [" " if type(x) == float else x for x in test_question1]
test_question1_vec = glove.transform(test_question1)

In [73]:
test_question2 = test[['question2_final']].to_numpy().tolist()
test_question2 = [item for sublist in test_question2 for item in sublist]
test_question2 = [" " if type(x) == float else x for x in test_question2]
test_question2_vec = glove.transform(test_question2)

In [78]:
test = test.drop(columns=['question1_final', 'question2_final'])
test = np.hstack((test_question1_vec, test_question2_vec))
test

array([[-0.07126475,  0.17173441,  0.02950917, ...,  0.09906823,
         0.27332693, -0.3322449 ],
       [ 0.5345375 ,  0.3970475 ,  0.04409725, ..., -0.20612001,
        -0.31522876, -0.10100287],
       [-0.2557675 ,  0.18423688,  0.21342987, ..., -0.2553483 ,
         0.18661287,  0.29054585],
       ...,
       [ 0.16415483,  0.48058942, -0.0047109 , ..., -0.25978884,
        -0.24332383, -0.36860567],
       [ 0.10923639,  0.27264813, -0.18366843, ..., -0.12674768,
        -0.44747126, -0.7042905 ],
       [ 0.5354652 ,  0.3937717 , -0.08833009, ..., -0.41618055,
        -0.15007499, -0.14127739]], dtype=float32)

In [75]:
y_pred = svc_clf_model.predict(test)

In [76]:
from sklearn.metrics import confusion_matrix, log_loss, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
print("Scores for best SVC with settings: ", svc_clf_model)
print(accuracy_score(y_pred, y_test))
print(precision_score(y_pred, y_test))
print(recall_score(y_pred, y_test))
print(log_loss(y_pred, y_test))
print(roc_auc_score(y_pred, y_test))
print(f1_score(y_pred, y_test))

Scores for best SVC with settings:  LinearSVC(C=100, dual=False, max_iter=3000)
0.6482475450790274
0.16175447260320097
0.5752834105061472
12.149345574570317
0.6159671160161228
0.25250985545335086
